In [6]:
link = "https://www.reuters.com/resources/archive/us/{}{}{}.html"
reuters_db = '/home/ostapkharysh/Documents/bt_data/DB/Reuters'

In [7]:
from bs4 import BeautifulSoup as BS
import requests
from requests.exceptions import HTTPError
#import re
import datetime
#import pytz
from newspaper import Article
from db_manager import *
import sqlite3
from tqdm import tqdm


years = ['2013']#, '2014', '2015', '2016', '2017']
months = ['01',]# '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = ['01', ]#'02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19'
       #'20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
    
    

In [11]:
conn = create_connection(reuters_db)

for y in years:
    for m in months:
        for d in days:
            try:
                datetime.datetime(year=int(y),month=int(m),day=int(d))
            except ValueError:
                print("Day doesn't exist")
                continue
            try:
                r = requests.get(link.format(y, m, d))
                r.raise_for_status() # check if page found
            except HTTPError:
                print("No page found by this link!",r.status)
                continue
                    
            soup = BS(r.content, 'html.parser')
            span_list = soup.find_all('div' , attrs={'class':"headlineMed"})

            header = str(soup.find_all('title')).split()
            day, month, year = header[6], header[7], header[8]

            k = [[str(el).split('>')[1].split("=")[1].replace('"', ''), str(el).split()[-2]] for el in span_list
                 if not str(el).split('>')[1].split("=")[1].replace('"', '').startswith('http://www.reuters.com/news/video')]
            
            #print(len(k))
            
            for link_time in k:
                try:
                    r = requests.get(link_time[0])
                    r.raise_for_status()
                    
                    article = Article(link_time[0].strip())
                    article.download()
                    article.parse()
                    title = article.title
                    text = article.text
                    time = link_time[1]
                    date = datetime.datetime.strptime("{} {} {} {}".format(month, day, year, time), '%b %d %Y %I:%M%p')

                    #print(str(date)+'\n'+title+'\n'+text+'\n'+link_time[0])
                    #print(str(date)+'\n'+title+'\n'+link_time[0])
                 
                    with conn:
                        news = (str(date), title, text, link_time[0])
                        create_task(conn, news, "Reuters_news")
                except HTTPError:
                    print("No article found by this link!", article.url)
                    
                    continue
                #print()
                
                
close_db(conn)

No article found by this link! http://www.reuters.com/article/2013/01/02/markets-singapore-stocksnews-property-idUSL4N0A70UG20130102
No article found by this link! http://www.reuters.com/article/2013/01/02/markets-ratings-sompo-idUSWNB269120130102
No article found by this link! http://www.reuters.com/article/2013/01/02/markets-hongkong-stocks-preopen-idUSL4N0A703720130102
No article found by this link! http://www.reuters.com/article/2013/01/02/fueloil-korea-wp-idUSL4N0A703520130102
No article found by this link! http://www.reuters.com/article/2013/01/01/usa-fiscal-strategy-idUSL1E9C135X20130101
No article found by this link! http://www.reuters.com/article/2013/01/01/us-obesity-paradox-idUSBRE9000BJ20130101
No article found by this link! http://www.reuters.com/article/2013/01/01/usa-shooting-connecticut-claim-idUSL1E9C124U20130101
No article found by this link! http://www.reuters.com/article/2013/01/01/us-usa-fiscalcliff-cbo-idUSBRE90008H20130101
No article found by this link! http://ww